<a href="https://colab.research.google.com/github/NNLogic/NNMPPublic/blob/main/%5BNNMP%5DNNSlide_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#説明 / Description
PDF/PNG/JPGからNNMP用のNNSlideデータを作成するスクリプトになります。

Betaテスト版です。

★[1]→[2]→[3]の順番に実行してください。

※NNMPに同梱されているUnityのスクリプトでは、crunch圧縮によりファイルサイズを小さくできます。

The script to create a NNSLide image file for NNMP from PDF/PNG/JPG:

"This is a beta test version.

★Please execute the steps in the following order: [1] → [2] → [3].

※The Unity script included in NNMP allows for file size reduction through crunch compression.

In [ ]:
# @title #変換設定 / Settings
#@markdown ↓ファイル名を表示する場合、未入力にしてください。/  If you want to display the file name, please leave it blank.
rename = "" #@param {type:"string"}
#@markdown ↓ファイル名の末尾番号をJsonの画像位置にする場合、"off"にしてください。/  If you want to use the file name's trailing number for the image position in the Json, set "off".
renumber = "on" #@param ["on", "off"]
#@markdown ↓スライドを変換する場合、max_widthは1024を推奨しています。/ When converting slides, we recommend using a max_width of 1024."
max_width = '2048' #@param ["1024", "2048", "4096", "8192", "Unlimited"]


#変換 / Convert

・Googleドライブからファイルをコピーする場合は、[0]を事前に実行してください。/ To copy a file from Google Drive, please execute [0] beforehand.


※ファイルサイズが大きい場合、Googleドライブから変換することを推奨します。/ If the file size is large, it is recommended to convert it from Google Drive.

In [ ]:
#@title [0] Googleドライブをマウントする / Mounting Google Drive
%cd  /content

import ipywidgets as widgets
from google.colab import drive

if 'drive' in locals():
  drive.flush_and_unmount()

drive.mount('/content/drive')

In [ ]:
#@title [1]　ファイルを読込 / Loading files
%cd  /content

load_type = "PNG, JPG" #@param ["PDF", "PNG, JPG"]
load_from = "Local Computer" #@param ["Google Drive", "Local Computer"]

#@markdown ↓Google Driveの場合、指定してください。/ In the case of Google Drive, please specify.
#@markdown 　PDFはファイルパス、PNG/JPGはフォルダパス / For PDF files, provide the file path, and for PNG/JPG files, provide the folder path.
drive_path = "/content/drive/MyDrive/NNMP Quick Guide.pdf" #@param {type:"string"}

if 'rename' not in locals():
    rename = ""
if 'max_width' not in locals():
    max_width = 1024

from pkgutil import extend_path
from google.colab import files
import os
import shutil

!rm -f *.pdf *.png *.jpg  *.dds *.crn *.json

def get_last_folder_path(path):
    path = path.rstrip(os.path.sep)
    dir_path, folder_name = os.path.split(path)
    return os.path.basename(folder_name)

if load_type ==  "PNG, JPG":
  image_files = []
  if load_from == "Local Computer":
    upload_files = files.upload()
    image_files = [file for file in upload_files if file.endswith('.png') or file.endswith('.jpg')]

  if load_from == "Google Drive":
    if not os.path.exists(drive_path):
      print("Error:drive_path is invalid")
      raise SystemExit
    copy_dest = "/content"
    os.chdir(drive_path)
    output_file_name =  get_last_folder_path(drive_path)

    image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]
    no=0
    for file in image_files:
      no = no + 1
      src_path = os.path.join(drive_path, file)
      dest_path = os.path.join(copy_dest, file)
      shutil.copy(src_path, dest_path)
    os.chdir("/content")
    image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]

  no=0
  for file in image_files:
    no = no + 1
    if rename != "":
      file_name, file_extension = os.path.splitext(file)
      new_name =  rename+"-"+str(no)+file_extension
      os.rename(file, os.path.join(os.path.dirname(file), new_name))

if load_type ==  "PDF":
  pdf_file_name =""
  base_file_name =""

  if load_from == "Local Computer":
    result = files.upload()
    pdf_file_name = list(result.keys())[0]
    base_file_name = os.path.splitext(pdf_file_name)[0]

  if load_from == "Google Drive":
    if not os.path.exists(drive_path):
      print("Error:drive_path is invalid")
      raise SystemExit
    copy_dest = "/content"  # コピー先のディレクトリパスを指定してください
    pdf_file_name = os.path.basename(drive_path)
    base_file_name = os.path.splitext(pdf_file_name)[0]
    output_file_name = base_file_name
    dest_path = os.path.join(copy_dest, pdf_file_name)
    shutil.copy(drive_path, dest_path)

  !apt-get install -y poppler-utils

  output_name = base_file_name if rename == "" else rename;

  if max_width == 'Unlimited':
    !pdftoppm -png "$pdf_file_name" "$output_name"
  else:
    !pdftoppm -scale-to $max_width -png "$pdf_file_name"  "$output_name"



In [ ]:
#@title [2]NNSlideデータへの変換 / Converting to NNSlide Data


!sudo apt-get install imagemagick
!pip install wand
!sudo sed -i 's/<policy domain="resource" name="memory" value="256MiB"\/>/<policy domain="resource" name="memory" value="10GiB"\/>/' /etc/ImageMagick-6/policy.xml

import os, sys
from wand.image import Image
import datetime
import base64
import json
import re
import subprocess

%cd  /content

#ASTC
filename="astcenc-4.7.0-linux-x64.zip"
if not os.path.isfile(filename):
    !wget https://github.com/ARM-software/astc-encoder/releases/download/4.7.0/astcenc-4.7.0-linux-x64.zip

!mkdir -p astc
!unzip -o astcenc-4.7.0-linux-x64.zip -d astc

def encode_astc(input_image, output_image, blocksize="6x6", quality="-medium"):
    command = [
        "astc/bin/astcenc-avx2", "-cl", input_image, output_image,
        blocksize, quality
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode == 0:
        print("Compression successful!")
    else:
        print("Compression failed:", result.stderr)

#ASTC


if 'max_width' not in locals():
    max_width = "1024"
if 'rename' not in locals():
    rename = ""
if 'renumber' not in locals():
    renumber = "on"

no = 0
images = []
byte_data = []
files = os.listdir("./")
def extract_number(file_name):
    match = re.search(r"\d+$", os.path.splitext(file_name)[0])
    return int(match.group()) if match else 0

def change_extension(file_name, new_extension):
    base_name = os.path.splitext(file_name)[0]  # 拡張子を除いたファイル名を取得
    new_file_name = f"{base_name}.{new_extension}"  # 新しいファイル名を組み立てる
    return new_file_name

sorted_files = sorted(files, key=extract_number)
for file_name in sorted_files:
    if file_name.endswith(".png") or file_name.endswith(".jpg"):

      im = Image(filename=file_name)




      if max_width != "Unlimited":
        if im.width > int(max_width):
          target_width = int(max_width)
          target_height = int(im.height * (target_width / im.width))
          im.resize(target_width, target_height)

      new_width = 4 * (im.width // 4)
      new_height = 4 * (im.height // 4)

      im.resize(new_width, new_height)

      im.flip()

      im.format = 'png'
      im.save(filename='astc/output.png')
      encode_astc("astc/output.png", "astc/output.astc","6x6")

      with open("astc/output.astc", 'rb') as file:
        file.seek(16)
        astc_data = file.read()

      b64_astc_txt = base64.b64encode(astc_data)
      type_astc = 50
      type_astc_str = "50"#6x6
               # TextureFormat.ASTC_4x4://48
                # TextureFormat.ASTC_5x5://49
                # TextureFormat.ASTC_6x6://50
                # TextureFormat.ASTC_8x8://51
                # TextureFormat.ASTC_10x10://52
                # TextureFormat.ASTC_12x12://53
      im.convert("RGBA")
      im.options['dds:mipmaps'] = "0"
      im.compression = "dxt1"
      im.format = 'dds'
      im_data = im.make_blob()[128:]
      b64_txt = base64.b64encode(im_data)

      #type_str = "10" if im.compression == "dxt1" else "10" # type DXT1 10 DXT5 10
      im_type = 10
      type_str = "10"#28 DXT1 crunch 29 DXT5 crunch
      if renumber == "off":
        match = re.search(r"\d+$", os.path.splitext(file_name)[0])
        no = int(match.group()) if match else no + 1
        no = 200 if no > 200 else 1 if no < 1 else no
      else:
         no = no + 1

##############################################
      image_dict = {"no":str(no),
                    "name":os.path.splitext(file_name)[0],
                    "type":type_str,
                    "type_astc":type_astc_str,
                    "width":str(im.width),
                    "height":str( im.height),
                    "base64":b64_txt.decode('utf-8'),
                    "base64_astc":b64_astc_txt.decode('utf-8')
                    };
      images.append(image_dict)
##############################################

      #DXT
      byte_image = bytearray(40)
      byte_name  =  image_dict["name"].encode('utf-8')
      byte_image += byte_name
      byte_image += im_data

      byte_image[0:4] = len(byte_image).to_bytes(4, byteorder='little')
      byte_image[4:8] =no.to_bytes(4, byteorder='little')
      byte_image[8:12] = im_type.to_bytes(4, byteorder='little')
      byte_image[24:28] =  im.width.to_bytes(4, byteorder='little')
      byte_image[28:32] =  im.height.to_bytes(4, byteorder='little')
      byte_image[32:36] = len(byte_name).to_bytes(4, byteorder='little')
      byte_image[36:40] = len(im_data).to_bytes(4, byteorder='little')

      #ASTC
      byte_image_astc = bytearray(40)
      byte_image_astc += byte_name
      byte_image_astc += astc_data

      byte_image_astc[0:4] = len(byte_image_astc).to_bytes(4, byteorder='little')
      byte_image_astc[4:8] = no.to_bytes(4, byteorder='little')
      byte_image_astc[8:12] = type_astc.to_bytes(4, byteorder='little')
      byte_image_astc[24:28] =  im.width.to_bytes(4, byteorder='little')
      byte_image_astc[28:32] =  im.height.to_bytes(4, byteorder='little')
      byte_image_astc[32:36] = len(byte_name).to_bytes(4, byteorder='little')
      byte_image_astc[36:40] = len(astc_data).to_bytes(4, byteorder='little')

      byte_data.append(byte_image)
      byte_data.append(byte_image_astc)

##############################################
dict= {"version":"001","images":images}
##############################################

version  = 1;
options  = 0;
byte_format = bytearray(20)
byte_name  =  image_dict["name"].encode('utf-8')

byte_format +=byte_name;

byte_format[0:4] = len(byte_format).to_bytes(4, byteorder='little')
byte_format[4:8] = version.to_bytes(4, byteorder='little')
byte_format[8:12] = options.to_bytes(4, byteorder='little')
byte_format[12:16] = int(len(byte_data)/2).to_bytes(4, byteorder='little')
byte_format[16:20] = len(byte_name).to_bytes(4, byteorder='little')

byte_format+= b''.join(byte_data)

byte_file_h = bytearray(14)
byte_file_h[0] = b'N'[0]
byte_file_h[1] = b'S'[0]

byte_file_h+=byte_format
byte_file_h[2:6] = len(byte_file_h).to_bytes(4, byteorder='little')
byte_data = byte_file_h
##############################################


# output
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d_%H%M%S")

if 'output_file_name' not in locals():
    output_file_name  = formatted_datetime

json_file_name = output_file_name + '.json'
with open(json_file_name, "w") as outputFile:
  json.dump(dict, outputFile, indent=2 )

nnslide_file_name = output_file_name + '.bin'
with open(nnslide_file_name, "wb") as outputFile:
  outputFile.write(byte_data)

In [ ]:
#@title [3]ファイルのダウンロード / Downloading the file
save_to = "Local Computer" #@param ["Google Drive", "Local Computer"]

from google.colab import files
import shutil

def create_directory_path(path):
    if os.path.isfile(path):
        path = os.path.dirname(path)
    if not path.endswith(os.path.sep):
        path += os.path.sep
    return path

if save_to == "Local Computer":
  files.download(json_file_name)
  files.download(nnslide_file_name)

if save_to == "Google Drive":
  shutil.copyfile(json_file_name, create_directory_path(drive_path)+"/"+json_file_name)


#その他 / Misc.


In [ ]:
#@title [Ex1]GitHub　URL変換 / Make GitHub URL
#@markdown GitHubのURLからNNPosterで使うURLに変換する / Convert the GitHub URL to the URL used by NNPoster.
github_url = "" #@param {type:"string"}

import re

def convert_github_url_to_raw(url):
    pattern = r"https://github.com/([^/]+)/([^/]+)/blob/([^/]+)/(.+)"
    replacement = r"https://raw.githubusercontent.com/\1/\2/\3/\4"
    raw_url = re.sub(pattern, replacement, url)
    return raw_url

raw_url = convert_github_url_to_raw(github_url)
print("GitHub URL":"+ raw_url)
